In [2]:
import time

start_time = time.time()
%run ./welding/predict.py
end_time = time.time()

print(f"Execution time: {end_time - start_time:.2f} seconds")

D:\LaserWeldMonitor\cv_postprocessing
D:\LaserWeldMonitor\cv_postprocessing\welding\examples\images\10a.jpg
[2025/08/26 15:36:07] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
v 1
l 392

WARNING  Model does not support 'augment=True', reverting to single-scale prediction.
0: 480x640 1 weld, 49.4ms
Speed: 4.1ms preprocess, 49.4ms inference, 144.6ms postprocess per image at shape (1, 3, 480, 640)

WARNING  Model does not support 'augment=True', reverting to single-scale prediction.
0: 480x640 1 weld, 7.4ms
Speed: 1.0ms preprocess, 7.4ms inference, 3.0ms postprocess per image at shape (1, 3, 480, 640)
t
 1121.0
le
 0.002551020408163265
T
 2.8596938775510203
{507: 'B', 515: 'B', 504: 'B', 509: 'B', 5011: 'C', 502: 'B'}
D:\LaserWeldMonitor\cv_postprocessing\welding\examples\images\10b.jpg
[2025/08/26 15:36:10] ppocr WARNING: Since the angle classifier is not initialized, it will not be used during the forward process
v 1
l 392



KeyboardInterrupt: 

Execution time: 83.20 seconds


In [2]:
from ultralytics import YOLO
import pycocotools
# Load your trained model
model = YOLO('welding/weights/main.pt')

# Run validation on the test set
metrics = model.val(
    data='data.yaml',
    split='val',        # use 'test' or 'val'
    iou=0.65,            # optional: change the default IoU threshold for NMS
    conf=0.001,          # optional: change the confidence threshold
    save_json=True,      # optional: saves results to a JSON file for later analysis
    plots=True           # optional: creates plots for confusion matrix, PR curves, etc.
)

# The results are stored in the 'metrics' object
print(f"mAP50-95: {metrics.box.map}")    # mAP@0.5:0.95
print(f"mAP50: {metrics.box.map50}")     # mAP@0.5
print(f"mAP75: {metrics.box.map75}")     # mAP@0.75

Ultralytics 8.3.67  Python-3.9.20 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
YOLOv8s-seg summary (fused): 195 layers, 11,779,987 parameters, 0 gradients, 42.4 GFLOPs


val: Scanning D:\LaserWeldMonitor\cv_postprocessing\welding\set\labels\val.cache... 231 images, 0 backgrounds, 0 corrup
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all        231        231          1      0.991      0.995      0.957          1      0.991      0.995      0.956
Speed: 0.6ms preprocess, 7.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving runs\segment\val7\predictions.json...
Results saved to runs\segment\val7
mAP50-95: 0.9568569126350497
mAP50: 0.995
mAP75: 0.981547329743483


In [1]:
!pip install pycocotools

In [4]:
import pandas as pd
import re

# Read the CSV file into a DataFrame
df = pd.read_csv('welding/examples/output/props.csv')

# Define a function to split the 'key' into numeric and alphabetical parts
def split_key(key):
    match = re.match(r"(\d+)([a-zA-Z]+)", key)
    if match:
        return (int(match.group(1)), match.group(2))
    return (float('inf'), '')  # ssshandle case where the key does not match the expected pattern

# Apply the function to create a sorting key
df['sort_key'] = df['key'].apply(split_key)

# Sort the DataFrame based on the sorting key
df = df.sort_values('sort_key')

# Drop the temporary 'sort_key' column
df = df.drop(columns=['sort_key'])

# Save the sorted DataFrame back to a CSV
df.to_csv('welding/examples/output/props.csv', index=False)


In [3]:
import os
import tarfile
import requests
from tqdm import tqdm
import urllib.request

def download_file(url, destination):
    """Download a file with progress bar"""
    print(f"Downloading {url} to {destination}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)
    
    # Download with progress
    urllib.request.urlretrieve(url, destination)
    print(f"Downloaded: {destination}")

def extract_tar(file_path, extract_to):
    """Extract tar file"""
    print(f"Extracting {file_path} to {extract_to}")
    os.makedirs(extract_to, exist_ok=True)
    
    with tarfile.open(file_path, 'r') as tar:
        tar.extractall(path=extract_to)
    print(f"Extracted: {file_path}")
    
    # Remove the tar file to save space
    os.remove(file_path)
    print(f"Removed: {file_path}")

def main():
    # Create models directory
    models_dir = os.path.join(os.getcwd(), 'models', 'paddleocr')
    os.makedirs(models_dir, exist_ok=True)
    
    # Model URLs
    models = {
        'det': 'https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar',
        'rec': 'https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar',
        'cls': 'https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar'
    }
    
    # Download and extract each model
    for model_type, url in models.items():
        tar_filename = os.path.join(models_dir, url.split('/')[-1])
        extract_dir = os.path.join(models_dir, url.split('/')[-1].replace('.tar', ''))
        
        # Download
        download_file(url, tar_filename)
        
        # Extract
        extract_tar(tar_filename, extract_dir)
    
    print("All models downloaded and extracted successfully!")
    print(f"Models are in: {models_dir}")

if __name__ == "__main__":
    main()

Downloaded: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv3_det_infer.tar
Extracting D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv3_det_infer.tar to D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv3_det_infer
Extracted: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv3_det_infer.tar
Removed: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv3_det_infer.tar
Downloaded: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv4_rec_infer.tar
Extracting D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv4_rec_infer.tar to D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv4_rec_infer
Extracted: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv4_rec_infer.tar
Removed: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_PP-OCRv4_rec_infer.tar
Downloaded: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\ch_ppocr_mobile_v2.0_cls_infer.tar
Extr

In [6]:
import os
import tarfile
import urllib.request

def download_file(url, destination):
    """Download a file with progress bar"""
    print(f"Downloading {url} to {destination}")
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(destination), exist_ok=True)
    
    # Download with progress
    urllib.request.urlretrieve(url, destination)
    print(f"Downloaded: {destination}")

def extract_tar(file_path, extract_to):
    """Extract tar file"""
    print(f"Extracting {file_path} to {extract_to}")
    os.makedirs(extract_to, exist_ok=True)
    
    with tarfile.open(file_path, 'r') as tar:
        tar.extractall(path=extract_to)
    print(f"Extracted: {file_path}")
    
    # Remove the tar file to save space
    os.remove(file_path)
    print(f"Removed: {file_path}")

def main():
    # Create models directory
    models_dir = os.path.join(os.getcwd(), 'models', 'paddleocr')
    os.makedirs(models_dir, exist_ok=True)
    
    # CPU-OPTIMIZED MODELS (Mobile versions - lighter and faster on CPU)
    # models = {
    #     'det': 'https://paddleocr.bj.bcebos.com/PP-OCRv2/chinese/ch_PP-OCRv2_det_infer.tar',
    #     'rec': 'https://paddleocr.bj.bcebos.com/PP-OCRv2/english/en_PP-OCRv2_rec_infer.tar',
    #     'cls': 'https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar'
    # }
    
    # Alternative: Even lighter English-only models
    models = {
        'det': 'https://paddleocr.bj.bcebos.com/dygraph_v2.0/multilingual/en_ppocr_mobile_v2.0_det_infer.tar',
        'rec': 'https://paddleocr.bj.bcebos.com/dygraph_v2.0/multilingual/en_number_mobile_v2.0_rec_infer.tar',
        'cls': 'https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar'
    }
    
    # Download and extract each model
    for model_type, url in models.items():
        tar_filename = os.path.join(models_dir, url.split('/')[-1])
        extract_dir = os.path.join(models_dir, url.split('/')[-1].replace('.tar', ''))
        
        # Download
        download_file(url, tar_filename)
        
        # Extract
        extract_tar(tar_filename, extract_dir)
    
    print("All CPU-optimized models downloaded and extracted successfully!")
    print(f"Models are in: {models_dir}")

if __name__ == "__main__":
    main()

Downloaded: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_ppocr_mobile_v2.0_det_infer.tar
Extracting D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_ppocr_mobile_v2.0_det_infer.tar to D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_ppocr_mobile_v2.0_det_infer
Extracted: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_ppocr_mobile_v2.0_det_infer.tar
Removed: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_ppocr_mobile_v2.0_det_infer.tar
Downloaded: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_number_mobile_v2.0_rec_infer.tar
Extracting D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_number_mobile_v2.0_rec_infer.tar to D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_number_mobile_v2.0_rec_infer
Extracted: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_number_mobile_v2.0_rec_infer.tar
Removed: D:\LaserWeldMonitor\cv_postprocessing\models\paddleocr\en_number_mobile_v2.0_rec_infer.tar
Downloaded: